# Baseline- correlation remover

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from fairlearn.preprocessing import CorrelationRemover
from sklearn.model_selection import KFold

In [ ]:
from src.data.unified_dataloader import load_dataset

a, processed_german_credit = load_dataset('german_credit')
_, processed_compas = load_dataset('compas')
_, processed_adult = load_dataset('adult')
_, processed_census_income_kdd = load_dataset('census_income_kdd')
_, processed_default_credit = load_dataset('default_credit')
_, processed_compas4race = load_dataset('compas4race')


Age                   0
Sex                   0
Job                   0
Housing               0
Saving accounts     183
Checking account    394
Credit amount         0
Duration              0
Purpose               0
Risk                  0
dtype: int64


### 1. German credit

In [5]:
processed_german_credit.head(3)

,Age,sex,Credit amount,Duration,Job_0,Job_1,Job_2,Job_3,Housing_free,Housing_own,...,Checking account_rich,Purpose_business,Purpose_car,Purpose_domestic appliances,Purpose_education,Purpose_furniture/equipment,Purpose_radio/TV,Purpose_repairs,Purpose_vacation/others,Risk
0,2.766456,0,-0.745131,-1.236478,0,0,1,0,0,1,...,0,0,0,0,0,0,1,0,0,0
1,-1.191404,1,0.949817,2.248194,0,0,1,0,0,1,...,0,0,0,0,0,0,1,0,0,1
2,1.183312,0,-0.416562,-0.738668,0,1,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0


In [6]:
df = processed_german_credit.copy()
X = df.drop('Risk', axis=1)
y = df['Risk']

In [7]:
def correlation_remover(original_X, sensitive_feature=['sex']):
    cr = CorrelationRemover(sensitive_feature_ids=sensitive_feature, alpha=0.5)
    cr.fit(original_X)
    # CorrelationRemover(sensitive_feature_ids=sensitive_feature)
    X_transform = cr.transform(original_X)
    return X_transform

In [8]:
from src.attribution.oracle_metric import perturb_numpy_ver
from fairness_measures import marginalised_np_mat, grp1_DP, grp2_EO, grp3_PQP

def fairness_value_function(sen_att, priv_val, unpriv_dict, X, model):
    X_disturbed = perturb_numpy_ver(
        X=X,
        sen_att=sen_att,
        priv_val=priv_val,
        unpriv_dict=unpriv_dict,
        ratio=1.0,
    )
    fx = model.predict_proba(X)[:, 1]
    fx_q = model.predict_proba(X_disturbed)[:, 1]
    return np.mean(np.abs(fx - fx_q))


In [11]:
from src.attribution import FairnessExplainer
kf = KFold(n_splits=5, shuffle=True, random_state=1)  # 5-fold 交叉验证
i = 1

accuracies_of_origin = []
accuracies_of_transform = []
DRs_of_origin = []
DRs_of_transform = []
EOs_of_origin = []
EOs_of_transform = []
DPs_of_origin = []
DPs_of_transform = []
PQPs_of_origin = []
PQPs_of_transform = []

for train_index, val_index in kf.split(X):
    # print("-------------------------------------")
    # print(f"-------------{i}th fold----------------")
    # print("-------------------------------------")
    X_train_fold, X_val_fold = X.iloc[train_index], X.iloc[val_index]
    y_train_fold, y_val_fold = y.iloc[train_index], y.iloc[val_index]

    model = XGBClassifier()
    # 训练模型
    model.fit(X_train_fold, y_train_fold)

    sen_att_name = ['sex']
    sen_att = [X_val_fold.columns.get_loc(name) for name in sen_att_name]
    priv_val = [1]
    unpriv_dict = [list(set(X_val_fold.values[:, sa])) for sa in sen_att]
    for sa_list, pv in zip(unpriv_dict, priv_val):
        sa_list.remove(pv)
    fairness_explainer_original = FairnessExplainer(
            model=model, 
            sen_att=sen_att, 
            priv_val=priv_val, 
            unpriv_dict=unpriv_dict,
            fairshap_base='DR'
            )
    # 预测
    y_val_pred = model.predict(X_val_fold)
    original_accuracy = accuracy_score(y_val_fold , y_val_pred)
    original_DR = fairness_value_function(sen_att, priv_val, unpriv_dict, X_val_fold.values, model)

    priv_idx = X_val_fold['sex'].to_numpy().astype(bool)
    g1_Cm, g0_Cm = marginalised_np_mat(y=y_val_fold, y_hat=y_val_pred, pos_label=1, priv_idx=priv_idx)
    original_DP = grp1_DP(g1_Cm, g0_Cm)[0]
    original_EO = grp2_EO(g1_Cm, g0_Cm)[0]
    original_PQP = grp3_PQP(g1_Cm, g0_Cm)[0]

    accuracies_of_origin.append(original_accuracy)
    DRs_of_origin.append(original_DR)
    DPs_of_origin.append(original_DP)
    EOs_of_origin.append(original_EO)
    PQPs_of_origin.append(original_PQP)
    
    ## transform
    sensitive_column = X_train_fold['sex'].copy()
    model_transform = XGBClassifier()
    X_train_transform_without_sen_att = correlation_remover(X_train_fold, sen_att_name)
    X_transform_with_sensitive = np.insert(X_train_transform_without_sen_att, 1, sensitive_column, axis=1)
    model_transform.fit(X_transform_with_sensitive, y_train_fold)
    y_val_transform_pred = model_transform.predict(X_val_fold)
    transform_accuracy = accuracy_score(y_val_fold , y_val_transform_pred)
    transform_DR = fairness_value_function(sen_att, priv_val, unpriv_dict, X_val_fold.values, model_transform)
    g1_Cm_transform, g0_Cm_transform = marginalised_np_mat(y=y_val_fold, y_hat=y_val_transform_pred, pos_label=1, priv_idx=priv_idx)
    transform_DP = grp1_DP(g1_Cm_transform, g0_Cm_transform)[0]
    transform_EO = grp2_EO(g1_Cm_transform, g0_Cm_transform)[0]
    transform_PQP = grp3_PQP(g1_Cm_transform, g0_Cm_transform)[0]

    accuracies_of_transform.append(transform_accuracy)
    DRs_of_transform.append(transform_DR)
    DPs_of_transform.append(transform_DP)
    EOs_of_transform.append(transform_EO)
    PQPs_of_transform.append(transform_PQP)

    i = i + 1

In [12]:
print("original_accuracy: ", np.mean(accuracies_of_origin))
print("original_DR: ", np.mean(DRs_of_origin))
print("original_DP: ", np.mean(DPs_of_origin))
print("original_EO: ", np.mean(EOs_of_origin))
print("original_PQP: ", np.mean(PQPs_of_origin))

print("transform_accuracy: ", np.mean(accuracies_of_transform))
print("transform_DR: ", np.mean(DRs_of_transform))
print("transform_DP: ", np.mean(DPs_of_transform))
print("transform_EO: ", np.mean(EOs_of_transform))
print("transform_PQP: ", np.mean(PQPs_of_transform))

original_accuracy:  0.6649999999999999
original_DR:  0.07850321
original_DP:  0.051212428151272006
original_EO:  0.12865149256676855
original_PQP:  0.13410967540351496
transform_accuracy:  0.6819999999999999
transform_DR:  0.0017339538
transform_DP:  0.05330876595452962
transform_EO:  0.149534537634666
transform_PQP:  0.1784764243265364


# 2. Compas

In [13]:
'''COMPAS'''
df = processed_compas.copy()
X = df.drop('two_year_recid', axis=1)
y = df['two_year_recid']

X.head(3)

,sex,age,juv_fel_count,juv_misd_count,juv_other_count,priors_count,race_African-American,race_Asian,race_Caucasian,race_Hispanic,race_Native American,race_Other,c_charge_degree_F,c_charge_degree_M,type_of_assessment_Risk of Recidivism,score_text_High,score_text_Low,score_text_Medium
0,1,2.875313,-0.141855,-0.187414,-0.218065,-0.711240,0,0,0,0,0,1,1,0,1,0,1,0
1,1,-0.068808,-0.141855,-0.187414,-0.218065,-0.711240,1,0,0,0,0,0,1,0,1,0,1,0
2,1,-0.909985,-0.141855,-0.187414,1.775750,0.108063,1,0,0,0,0,0,1,0,1,0,1,0


In [20]:
from src.attribution import FairnessExplainer
kf = KFold(n_splits=5, shuffle=True, random_state=1)  # 5-fold 交叉验证
i = 1

accuracies_of_origin = []
accuracies_of_transform = []
DRs_of_origin = []
DRs_of_transform = []
EOs_of_origin = []
EOs_of_transform = []
DPs_of_origin = []
DPs_of_transform = []
PQPs_of_origin = []
PQPs_of_transform = []

for train_index, val_index in kf.split(X):
    # print("-------------------------------------")
    # print(f"-------------{i}th fold----------------")
    # print("-------------------------------------")
    X_train_fold, X_val_fold = X.iloc[train_index], X.iloc[val_index]
    y_train_fold, y_val_fold = y.iloc[train_index], y.iloc[val_index]

    model = XGBClassifier()
    # 训练模型
    model.fit(X_train_fold, y_train_fold)

    sen_att_name = ['sex']
    sen_att = [X_val_fold.columns.get_loc(name) for name in sen_att_name]
    priv_val = [1]
    unpriv_dict = [list(set(X_val_fold.values[:, sa])) for sa in sen_att]
    for sa_list, pv in zip(unpriv_dict, priv_val):
        sa_list.remove(pv)
    fairness_explainer_original = FairnessExplainer(
            model=model, 
            sen_att=sen_att, 
            priv_val=priv_val, 
            unpriv_dict=unpriv_dict,
            fairshap_base='DR'
            )
    # 预测
    y_val_pred = model.predict(X_val_fold)
    original_accuracy = accuracy_score(y_val_fold , y_val_pred)
    original_DR = fairness_value_function(sen_att, priv_val, unpriv_dict, X_val_fold.values, model)

    priv_idx = X_val_fold['sex'].to_numpy().astype(bool)
    g1_Cm, g0_Cm = marginalised_np_mat(y=y_val_fold, y_hat=y_val_pred, pos_label=1, priv_idx=priv_idx)
    original_DP = grp1_DP(g1_Cm, g0_Cm)[0]
    original_EO = grp2_EO(g1_Cm, g0_Cm)[0]
    original_PQP = grp3_PQP(g1_Cm, g0_Cm)[0]

    accuracies_of_origin.append(original_accuracy)
    DRs_of_origin.append(original_DR)
    DPs_of_origin.append(original_DP)
    EOs_of_origin.append(original_EO)
    PQPs_of_origin.append(original_PQP)
    
    ## transform
    sensitive_column = X_train_fold['sex'].copy()
    model_transform = XGBClassifier()
    X_train_transform_without_sen_att = correlation_remover(X_train_fold, sen_att_name)
    X_transform_with_sensitive = np.insert(X_train_transform_without_sen_att, 0, sensitive_column, axis=1)
    model_transform.fit(X_transform_with_sensitive, y_train_fold)
    y_val_transform_pred = model_transform.predict(X_val_fold)
    transform_accuracy = accuracy_score(y_val_fold , y_val_transform_pred)
    transform_DR = fairness_value_function(sen_att, priv_val, unpriv_dict, X_val_fold.values, model_transform)
    g1_Cm_transform, g0_Cm_transform = marginalised_np_mat(y=y_val_fold, y_hat=y_val_transform_pred, pos_label=1, priv_idx=priv_idx)
    transform_DP = grp1_DP(g1_Cm_transform, g0_Cm_transform)[0]
    transform_EO = grp2_EO(g1_Cm_transform, g0_Cm_transform)[0]
    transform_PQP = grp3_PQP(g1_Cm_transform, g0_Cm_transform)[0]

    accuracies_of_transform.append(transform_accuracy)
    DRs_of_transform.append(transform_DR)
    DPs_of_transform.append(transform_DP)
    EOs_of_transform.append(transform_EO)
    PQPs_of_transform.append(transform_PQP)

    i = i + 1

In [23]:
print("original_accuracy: ", np.mean(accuracies_of_origin))
print("original_DR: ", np.mean(DRs_of_origin))
print("original_DP: ", np.mean(DPs_of_origin))
print("original_EO: ", np.mean(EOs_of_origin))
print("original_PQP: ", np.mean(PQPs_of_origin))

print("transform_accuracy: ", np.mean(accuracies_of_transform))
print("transform_DR: ", np.mean(DRs_of_transform))
print("transform_DP: ", np.mean(DPs_of_transform))
print("transform_EO: ", np.mean(EOs_of_transform))
print("transform_PQP: ", np.mean(PQPs_of_transform))

original_accuracy:  0.6698091989354125
original_DR:  0.08830647
original_DP:  0.1548324663847108
original_EO:  0.12428469251342018
original_PQP:  0.04916781015470504
transform_accuracy:  0.6698070843701911
transform_DR:  0.005672279
transform_DP:  0.11084330858040499
transform_EO:  0.06713923474962401
transform_PQP:  0.07494560579537339
